In [1]:
import os

In [2]:
%pwd

'a:\\Machine-Learning-Project-with-MLflow\\research'

In [3]:
os.listdir(os.getcwd())

['1.data_ingestion.ipynb',
 '2.data_validation.ipynb',
 '3.data_transformation.ipynb',
 '4.model_trainer.ipynb',
 '5.model_evaluation.ipynb',
 'trials.ipynb']

In [14]:
os.chdir("../")

In [15]:
%pwd

'a:\\Machine-Learning-Project-with-MLflow'

In [18]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rutvik46/Machine-Learning-Project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rutvik46" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="fdf9d63cdff88d1ed6fd22bc2f3d34a8c0672d1a"

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:str
    test_data_path:Path
    model_path:Path
    all_params:dict
    metrics_file_path:Path
    target_column:str
    mlflow_url:str

In [20]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        print(f"Config file path: {config_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_evaluation.root_dir])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        param = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=param,
            metrics_file_path=Path(config.metrics_file_path),
            target_column=schema.name,
            mlflow_url="https://dagshub.com/Rutvik46/Machine-Learning-Project-with-MLflow.mlflow"
        )

        return model_evaluation_config

In [22]:
import pandas as pd
from ml_project import logger
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from ml_project.utils.common import save_json 


In [28]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluation_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        r2 = r2_score(actual, pred)
        mae = mean_absolute_error(actual, pred)

        return rmse, r2, mae

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.evaluation_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metrics_file_path), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    logger.exception(e)
    raise e

Config file path: config\config.yaml
[2025-07-19 17:33:26,160 : INFO: yaml file: config\config.yaml loaded successfully]
[2025-07-19 17:33:26,162 : INFO: yaml file: params.yaml loaded successfully]
[2025-07-19 17:33:26,165 : INFO: yaml file: schema.yaml loaded successfully]
[2025-07-19 17:33:26,166 : INFO: created directory at: artifacts/model_evaluation]
[2025-07-19 17:33:26,333 : INFO: json file saved at: artifacts\model_evaluation\metrics.json]


C:\Users\rutvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.7.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\rutvi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2025/07/19 17:33:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Cr